In [1]:
# basic (built-in) Python packages
import numpy as np
from numpy.linalg import norm
import pandas as pd
import matplotlib.pyplot as plt

# advanced (built-in) Python packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize

# my implemented Python functions and classes
from data.generate_data import generate_data
from model.BaseModel import BaseModel
from model.Initial import Initial
# from model.MLE import MLE
from model.OS import OS
from model.MS import MS
from model.ORACLE_beta import ORACLE_beta
from model.ORACLE_sigma import ORACLE_sigma

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def compute_rmse(est, true):
    est = est.ravel()
    true = true.ravel()
    length = len(est)
    rmse = norm(est - true) / np.sqrt(length)
    return rmse

# Hyperparameters

In [3]:
seed = 1
np.random.seed(seed=seed)

N = 5000        # the size of the unlabeled dataset
r = 0.5
n = int(N * r)            # pilot sample size
alpha = 1 # n**(-0.1)  
print(f"alpha={alpha:.4f}")

p = 20          # feature dimension
K = 2           # (K+1) classes
M = int(0.5 * int(n / np.log(n)))          # the size of the annotator pool #int(n / np.log(n))
print(f"[n*alpha={int(n*alpha)}/N={N}]")
print(f"[M={M}] vs [n*alpha={int(n*alpha)}]")

alpha=1.0000
[n*alpha=2500/N=5000]
[M=159] vs [n*alpha=2500]


# Data Generation

In [4]:
beta, sigma, theta, X, Y, X1, X2, Y1, Y2, A1, AY1 = generate_data(K, p, N, n, M, alpha, seed=0)

True Labels 2    1836
1    1619
0    1545
dtype: int64 



In [5]:
sigma

array([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25,
       0.25, 0.25, 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  ,
       4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.  , 4.

# Initial Estimator

In [6]:
init_model = Initial(X1, AY1, A1, K)
init_beta, init_sigma, init_betams = init_model.init_param()

In [7]:
init_beta_rmse = compute_rmse(init_beta, beta[1:])
init_sigma_rmse = compute_rmse(init_sigma, sigma)
print(f"Init beta:  {init_beta_rmse:.7f}")
print(f"Init sigma: {init_sigma_rmse:.7f}")

Init beta:  0.0047335
Init sigma: 0.4963081


In [8]:
# diff_mom, diff_son = init_model.check(init_beta, init_sigma, beta[1:], sigma)
# plt.boxplot([diff_mom, diff_son.ravel()])
# plt.show()

# Oracle_beta

In [9]:
orab_model = ORACLE_beta(X1, AY1, A1, K, init_beta, sigma)
oracle_beta = orab_model.update_alg(max_steps=5, tol=1e-5, true_beta=beta[1:])

######## [Step 1] ########
norm(gradient): 9.2086297
RMSE(beta): 0.0097362
######## [Step 2] ########
norm(gradient): 0.1784264
RMSE(beta): 0.0077418
######## [Step 3] ########
norm(gradient): 0.0006958
RMSE(beta): 0.0077368
######## [Step 4] ########
norm(gradient): 0.0000000
RMSE(beta): 0.0077368


In [10]:
oracle_beta_rmse = compute_rmse(oracle_beta, beta[1:])
print(f"Oracle beta: {oracle_beta_rmse:.7f}")

Oracle beta: 0.0011830


In [11]:
# diff_mom, diff_son = orabeta_model.check(beta[1:], sigma)
# plt.boxplot([diff_mom, diff_son.ravel()])
# plt.show()

# Oracle_sigma

In [12]:
oras_model = ORACLE_sigma(X1, AY1, A1, K, beta[1:], init_sigma)
oracle_sigma = oras_model.update_alg(max_steps=5, tol=1e-5, true_sigma=sigma)

######## [Step 1] ########
norm(sigma): 0.2506866
RMSE(sigma): 3.2447190
######## [Step 2] ########
norm(sigma): 0.0191161
RMSE(sigma): 2.8736129
######## [Step 3] ########
norm(sigma): 0.0026310
RMSE(sigma): 3.1564271
######## [Step 4] ########
norm(sigma): 0.0004343
RMSE(sigma): 3.2201256
######## [Step 5] ########
norm(sigma): 0.0000378
RMSE(sigma): 3.2239692


In [13]:
oracle_sigma_rmse = compute_rmse(oracle_sigma, sigma)
print(f"Oracle sigma: {oracle_sigma_rmse:.7f}")

Oracle sigma: 0.2556774


# One Step Update

In [14]:
os_model = OS(X1, AY1, A1, K, init_beta, init_sigma)
os_beta, os_sigma = os_model.update_alg(max_steps=1, tol=1e-5, true_beta=beta[1:].ravel())

######## [Step 1] ########
norm(gradient): 9.2476085
RMSE(beta): 0.0075051


In [15]:
ts_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
ts_beta, ts_sigma = ts_model.update_alg(max_steps=2, tol=1e-5, true_beta=beta[1:].ravel())

######## [Step 1] ########
norm(gradient): 0.2832383
RMSE(beta): 0.0074516
######## [Step 2] ########
norm(gradient): 0.0184799
RMSE(beta): 0.0074692


In [16]:
ms_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
ms_beta, ms_sigma = ms_model.update_alg(max_steps=3, tol=1e-5, true_beta=beta[1:].ravel())

######## [Step 1] ########
norm(gradient): 0.0071724
RMSE(beta): 0.0074692
######## [Step 2] ########
norm(gradient): 0.0005303
RMSE(beta): 0.0074695
######## [Step 3] ########
norm(gradient): 0.0000886
RMSE(beta): 0.0074695


In [17]:
os_beta_rmse = compute_rmse(os_beta, beta[1:])
os_sigma_rmse = compute_rmse(os_sigma, sigma)
print(f"OS beta: {os_beta_rmse:.7f}")
print(f"OS sigma: {os_sigma_rmse:.7f}")

OS beta: 0.0011810
OS sigma: 0.2545334


In [18]:
ts_beta_rmse = compute_rmse(ts_beta, beta[1:])
ts_sigma_rmse = compute_rmse(ts_sigma, sigma)
print(f"TS beta: {ts_beta_rmse:.7f}")
print(f"TS sigma: {ts_sigma_rmse:.7f}")

TS beta: 0.0011810
TS sigma: 0.2545334


In [19]:
ms_beta_rmse = compute_rmse(ms_beta, beta[1:])
ms_sigma_rmse = compute_rmse(ms_sigma, sigma)
print(f"MS beta: {ms_beta_rmse:.7f}")
print(f"MS sigma: {ms_sigma_rmse:.7f}")

MS beta: 0.0011810
MS sigma: 0.2545334


In [20]:
# diff_mom, diff_son, var = os_model.check(init_beta, init_sigma, beta[1:], sigma)
# plt.boxplot([diff_mom.ravel(), diff_son.ravel()])
# plt.show()

- results compare:

In [21]:
# beta
print(f"Init beta: {init_beta_rmse:.7f}")
print(f"Ora  beta: {oracle_beta_rmse:.7f}")
print(f"OS   beta: {os_beta_rmse:.7f}")
print(f"TS   beta: {ts_beta_rmse:.7f}")
print(f"MS   beta: {ms_beta_rmse:.7f}")

# sigma
print(f"Init sigma: {init_sigma_rmse:.7f}")
print(f"Ora  sigma: {oracle_sigma_rmse:.7f}")
print(f"OS   sigma: {os_sigma_rmse:.7f}")
print(f"TS   sigma: {ts_sigma_rmse:.7f}")
print(f"MS   sigma: {ms_sigma_rmse:.7f}")

Init beta: 0.0047335
Ora  beta: 0.0011830
OS   beta: 0.0011810
TS   beta: 0.0011810
MS   beta: 0.0011810
Init sigma: 0.4963081
Ora  sigma: 0.2556774
OS   sigma: 0.2545334
TS   sigma: 0.2545334
MS   sigma: 0.2545334


# Compare

In [21]:
# N = 10000                        # the size of the unlabeled dataset
p = 20                           # feature dimension
K = 2                            # (K+1) classes
r = 0.5

In [22]:
B = 1000
RMSE_results = []
NAME_results = []

In [23]:
alpha_list = [0.5, 1.0]
N_list = [5000, 10000, 15000]

In [24]:
for N in N_list:
    n = int(N * r)                 # pilot sample size
    M = int(0.5 * n / np.log(n))   # the size of the annotator pool
    for alpha in alpha_list:
        print(f"[n*alpha={int(n*alpha)}/N={N}]")
        print(f"[M={M}] vs [n*alpha={int(n*alpha)}]")
        for seed in range(B):
            RMSE_list = [seed, n, alpha, M]
            NAME_list = ["seed", "n", "alpha", "M"]
            np.random.seed(seed)
            beta, sigma, theta, X, Y, X1, X2, Y1, Y2, A1, AY1 = generate_data(K, p, N, n, M, alpha, seed=seed)
        
            # Initial Estimator
            print(f"\n########## Initial Estimator ##########")
            init_model = Initial(X1, AY1, A1, K)
            init_beta, init_sigma, init_betams = init_model.init_param()
            init_beta_rmse = compute_rmse(init_beta, beta[1:])
            init_sigma_rmse = compute_rmse(init_sigma, sigma)
            RMSE_list += [init_beta_rmse, init_sigma_rmse]
            NAME_list += ["init_beta", "init_sigma"]
            
            # OS (One-Step) Estimator 
            print(f"\n########## OS Estimator ##########")
            os_model = OS(X1, AY1, A1, K, init_beta, init_sigma)
            os_beta, os_sigma = os_model.update_alg(max_steps=1, tol=1e-5, true_beta=beta[1:].ravel())
            os_beta_rmse = compute_rmse(os_beta, beta[1:])
            os_sigma_rmse = compute_rmse(os_sigma, sigma)
            RMSE_list += [os_beta_rmse, os_sigma_rmse]
            NAME_list += ["os_beta", "os_sigma"]
        
            # TS
            print(f"\n########## TS Estimator ##########")
            ts_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
            ts_beta, ts_sigma = ts_model.update_alg(max_steps=2, tol=1e-5, true_beta=beta[1:].ravel())
            ts_beta_rmse = compute_rmse(ts_beta, beta[1:])
            ts_sigma_rmse = compute_rmse(ts_sigma, sigma)
            RMSE_list += [ts_beta_rmse, ts_sigma_rmse]
            NAME_list += ["ts_beta", "ts_sigma"]
        
            # MS
            print(f"\n########## MS Estimator ##########")
            ms_model = MS(X1, AY1, A1, K, init_beta, init_sigma)
            ms_beta, ms_sigma = ms_model.update_alg(max_steps=3, tol=1e-5, true_beta=beta[1:].ravel())
            ms_beta_rmse = compute_rmse(ms_beta, beta[1:])
            ms_sigma_rmse = compute_rmse(ms_sigma, sigma)
            RMSE_list += [ms_beta_rmse, ms_sigma_rmse]
            NAME_list += ["ms_beta", "ms_sigma"]
        
            # Oracle_beta
            print(f"\n########## ORACLE_beta Estimator ##########")
            oracle_model = ORACLE_beta(X1, AY1, A1, K, init_beta, sigma)
            oracle_beta = oracle_model.update_alg(max_steps=5, tol=1e-5, true_beta=beta[1:])
            oracle_beta_rmse = compute_rmse(oracle_beta, beta[1:])
            oracle_model = ORACLE_sigma(X1, AY1, A1, K, beta[1:], init_sigma)
            oracle_sigma = oracle_model.update_alg(max_steps=5, tol=1e-5, true_sigma=sigma)
            oracle_sigma_rmse = compute_rmse(oracle_sigma, sigma)
            RMSE_list += [oracle_beta_rmse, oracle_sigma_rmse]
            NAME_list += ["oracle_beta", "oracle_sigma"]
        
            # Print Results
            # beta
            print()
            print(f"Init beta: {init_beta_rmse:.7f}")
            print(f"Ora  beta: {oracle_beta_rmse:.7f}")
            print(f"OS   beta: {os_beta_rmse:.7f}")
            print(f"TS   beta: {ts_beta_rmse:.7f}")
            print(f"MS   beta: {ms_beta_rmse:.7f}")
            print()
            # sigma
            print(f"Init sigma: {init_sigma_rmse:.7f}")
            print(f"Ora  sigma: {oracle_sigma_rmse:.7f}")
            print(f"OS   sigma: {os_sigma_rmse:.7f}")
            print(f"TS   sigma: {ts_sigma_rmse:.7f}")
            print(f"MS   sigma: {ms_sigma_rmse:.7f}")
        
            # Record Results
            RMSE_results.append(RMSE_list)
            a = pd.DataFrame(RMSE_results, columns=NAME_list)
            a.to_csv(f"/Users/helenology/Desktop/rmse_data.csv")

[n*alpha=1250/N=5000]
[M=159] vs [n*alpha=1250]
True Labels 2    1836
1    1619
0    1545
dtype: int64 


########## Initial Estimator ##########

########## OS Estimator ##########
######## [Step 1] ########
norm(gradient): 5.6001977
RMSE(beta): 0.0210615

########## TS Estimator ##########
######## [Step 1] ########
norm(gradient): 5.6001977
RMSE(beta): 0.0210615
######## [Step 2] ########
norm(gradient): 0.6038384
RMSE(beta): 0.0176278

########## MS Estimator ##########
######## [Step 1] ########
norm(gradient): 5.6001977
RMSE(beta): 0.0210615
######## [Step 2] ########
norm(gradient): 0.6038384
RMSE(beta): 0.0176278
######## [Step 3] ########
norm(gradient): 0.2497650
RMSE(beta): 0.0176680

########## ORACLE_beta Estimator ##########
######## [Step 1] ########
norm(gradient): 5.2987152
RMSE(beta): 0.0147143
######## [Step 2] ########
norm(gradient): 0.1423314
RMSE(beta): 0.0147803
######## [Step 3] ########
norm(gradient): 0.0009509
RMSE(beta): 0.0147954
######## [Step 4] ########

KeyboardInterrupt: 